## Pose Module

In [6]:
import cv2 as cv
import mediapipe as mp
import time


class poseDetector():

    def __init__(self, mode = False, upBody = False,modelComplexity= 1, smooth = True , detectionCon = 0.5, trackCon = 0.5): 
        self.mode = mode
        self.upBody = upBody
        self.modelComplexity = modelComplexity
        self.smooth = smooth
        self.detectionCon = detectionCon
        self.trackCon = trackCon

        self.mpPose = mp.solutions.pose
        self.pose = self.mpPose.Pose(self.mode, self.upBody, self.modelComplexity, self.smooth, self.detectionCon, self.trackCon)
        self.mpDraw = mp.solutions.drawing_utils

    def findPose(self, img, draw = True):
        imgRGB = cv.cvtColor((img), cv.COLOR_BGR2RGB)
        self.results = self.pose.process(imgRGB)
        if self.results.pose_landmarks:
            if draw:
                # for poselmrk in self.results.pose_landmarks:
                self.mpDraw.draw_landmarks(img, self.results.pose_landmarks, self.mpPose.POSE_CONNECTIONS)
        return img

    def findPosition(self, img, draw = True):
        lmList = []
        if self.results.pose_landmarks:
            myPose = self.results.pose_landmarks
            for id, lm in enumerate(myPose.landmark):
                # print(id,lm)
                h, w, c = img.shape 
                cx, cy = int(lm.x*w), int(lm.y*h)
                lmList.append([id, cx, cy])
                if draw:
                    cv.circle(img,(cx,cy), 5 , (0,150,150), cv.FILLED) 
        return lmList


def main():
    
    # Dummy Code
    
    pTime = 0
    cap = cv.VideoCapture(0)
    detector = poseDetector()
    while True:
        isTrue, frame = cap.read()
        img = cv.resize(frame, (640,480))

        img = detector.findPose(img, draw = True)
        lmList = detector.findPosition(img, draw = True)
        
        # lmList[P] = [P x y], Access particular Point
        # where P is Pose landmark point and x,y are coordinates of the point of that particular image or frame
        
        if len(lmList) != 0:
            cv.circle(img,(lmList[0][1],lmList[0][2]), 10 , (180,120,150), cv.FILLED) 
        
        # FPS
        cTime = time.time()
        fps = 1/(cTime-pTime)
        pTime = cTime
        cv.putText(img, str(int(fps)), (5, 20), cv.FONT_HERSHEY_COMPLEX, 0.5, (255,255,255), 1)

        cv.imshow("webCam", img)
        if cv.waitKey(1) & 0xFF == ord('x'):
            break
        
    cap.release()
    cv.destroyAllWindows()


if __name__ == "__main__":
    main()